In [1]:
from threaded_batching import ThreadedBatcher

In [2]:
class TestBatcher(ThreadedBatcher):
    
    def __init__(self, num_batches=100, **kwargs):
        self._next_batch_idx = 0
        self._num_batches = num_batches
        super(TestBatcher, self).__init__(**kwargs)
    
    def _reset_batch_params(self):
        self._next_batch_idx = 0
    
    def _get_next_batch_params(self):
        if self._next_batch_idx < self._num_batches:
            self._next_batch_idx += 1
            return (self._next_batch_idx-1)
        else:
            return None
    
    def _get_batch_from_params(self, params):
        return params

In [3]:
tds = TestBatcher()

In [4]:
epoch = [batch for batch in tds]
set(epoch) == set(range(100))

True

In [5]:
epoch = [batch for batch in tds]
set(epoch) == set(range(100))

True